In [1]:
import pandas as pd
import datetime
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

from sqlalchemy import create_engine

In [ ]:
#connection to a PostgreSQL local database
# ===========================================

# user_name_var = input('What is the username of your database? ')
host_var = input('What is the host address of your database?')
pw = input("Enter the database password: ")
# db_name = input("Enter the name of the database: ")

string_output = "postgresql://postgres"+':'+pw+'@'+host_var+":5432"+'/earthquake_db'
engine = create_engine(string_output)
connection = engine.connect()
print("Connected to database...")

In [3]:
engine = create_engine(string_output)
connection = engine.connect()
print("Connected to database...")

Connected to database...


Adding 7 days of data

In [4]:
page_url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"

df = pd.read_csv(page_url)
df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-22T02:28:31.120Z,33.423667,-117.152167,0.690000,1.36,ml,10.0,78.00,0.289600,0.3400,...,2022-01-22T02:32:09.645Z,"8km S of Temecula, CA",earthquake,1.08,31.61,0.373,16.0,automatic,ci,ci
1,2022-01-22T02:20:20.390Z,19.839500,-155.359665,30.150000,2.18,md,25.0,137.00,NaN,0.1500,...,2022-01-22T02:23:19.850Z,"20 km SW of Laupāhoehoe, Hawaii",earthquake,0.82,1.37,1.080,3.0,automatic,hv,hv
2,2022-01-22T01:54:41.810Z,19.212500,-155.417999,33.220001,2.08,md,37.0,143.00,NaN,0.1200,...,2022-01-22T01:57:52.220Z,"6 km E of Pāhala, Hawaii",earthquake,0.71,1.08,1.420,20.0,automatic,hv,hv
3,2022-01-22T01:48:17.879Z,64.614800,-152.234900,0.000000,2.20,ml,NaN,NaN,NaN,1.0800,...,2022-01-22T01:57:21.344Z,"62 km S of Tanana, Alaska",earthquake,NaN,0.40,NaN,NaN,automatic,ak,ak
4,2022-01-22T01:45:18.320Z,38.789665,-122.762833,0.490000,1.42,md,22.0,66.00,0.011210,0.0600,...,2022-01-22T02:33:11.401Z,"2km NNW of The Geysers, CA",earthquake,0.23,0.38,0.080,6.0,automatic,nc,nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9245,2021-12-23T02:51:59.969Z,38.466100,-119.473500,9.500000,0.60,ml,10.0,74.27,0.047000,0.1681,...,2021-12-23T17:59:12.538Z,"5 km S of Walker, California",earthquake,NaN,1.50,0.130,2.0,reviewed,nn,nn
9246,2021-12-23T02:51:19.225Z,69.900300,-143.702300,32.100000,1.50,ml,NaN,NaN,NaN,0.7700,...,2022-01-21T07:38:05.589Z,"26 km S of Kaktovik, Alaska",earthquake,NaN,0.20,NaN,NaN,reviewed,ak,ak
9247,2021-12-23T02:51:07.919Z,36.102833,-97.838667,6.190000,1.23,ml,75.0,39.00,0.116977,0.2600,...,2022-01-03T15:37:55.724Z,"5 km E of Hennessey, Oklahoma",earthquake,NaN,0.50,0.240,29.0,reviewed,ok,ok
9248,2021-12-23T02:47:57.770Z,33.275167,-116.259500,4.670000,2.53,ml,109.0,16.00,0.082380,0.2100,...,2022-01-19T03:31:14.040Z,"11km ENE of Borrego Springs, CA",earthquake,0.12,0.54,0.133,113.0,reviewed,ci,ci


In [5]:
df.to_sql('earthquake_raw', engine, if_exists='append', index = False)

In [6]:
df.to_csv("csv/earthquake_raw.csv",index=False)

Load from database to update with new rows

In [7]:
model_df = pd.read_sql_table(
    "earthquake_raw",
    con=engine
)
model_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.875000,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.119000,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17819,2021-12-23T02:51:59.969Z,38.466100,-119.473500,9.50,0.60,ml,10.0,74.27,0.047000,0.1681,...,2021-12-23T17:59:12.538Z,"5 km S of Walker, California",earthquake,NaN,1.50,0.130,2.0,reviewed,nn,nn
17820,2021-12-23T02:51:19.225Z,69.900300,-143.702300,32.10,1.50,ml,NaN,NaN,NaN,0.7700,...,2022-01-21T07:38:05.589Z,"26 km S of Kaktovik, Alaska",earthquake,NaN,0.20,NaN,NaN,reviewed,ak,ak
17821,2021-12-23T02:51:07.919Z,36.102833,-97.838667,6.19,1.23,ml,75.0,39.00,0.116977,0.2600,...,2022-01-03T15:37:55.724Z,"5 km E of Hennessey, Oklahoma",earthquake,NaN,0.50,0.240,29.0,reviewed,ok,ok
17822,2021-12-23T02:47:57.770Z,33.275167,-116.259500,4.67,2.53,ml,109.0,16.00,0.082380,0.2100,...,2022-01-19T03:31:14.040Z,"11km ENE of Borrego Springs, CA",earthquake,0.12,0.54,0.133,113.0,reviewed,ci,ci


In [8]:
boolean = model_df.duplicated(subset=['id']).any()
print(boolean)

True


In [9]:
model_df.drop_duplicates(subset=['id'], inplace=True)
print(model_df)

                           time  latitude  longitude   depth  mag magType  \
0      2022-01-06T01:42:39.855Z   37.3840  -117.1075    1.10  1.3      ml   
1      2022-01-06T01:31:33.698Z   38.1810  -117.8180    7.30  1.4      ml   
2      2022-01-06T01:24:20.083Z   61.3407  -149.8852   30.00  1.5      ml   
3      2022-01-06T01:15:15.851Z   62.8703  -150.7625  111.30  2.1      ml   
4      2022-01-06T01:10:44.629Z   64.0145  -148.7657    1.90  1.5      ml   
...                         ...       ...        ...     ...  ...     ...   
17812  2021-12-23T03:08:37.279Z   63.5339  -148.0043    1.60  0.7      ml   
17814  2021-12-23T03:02:42.557Z  -57.3553   -25.8404   35.00  4.6      mb   
17816  2021-12-23T03:01:30.630Z    5.8044   124.0274  517.23  4.3      mb   
17817  2021-12-23T02:55:51.833Z  -10.3928   166.1567  131.16  4.5      mb   
17820  2021-12-23T02:51:19.225Z   69.9003  -143.7023   32.10  1.5      ml   

        nst     gap   dmin     rms  ...                   updated  \
0     

In [10]:
model_df.to_sql('earthquake_raw', engine, if_exists='replace', index = False)

In [11]:
model_df.to_csv("csv/earthquake_raw.csv",index=False)

In [12]:
model_df['time'] = pd.to_datetime(model_df['time'])
model_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06 01:42:39.855000+00:00,37.3840,-117.1075,1.10,1.3,ml,8.0,149.46,0.875,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.1,0.290,4.0,automatic,nn,nn
1,2022-01-06 01:31:33.698000+00:00,38.1810,-117.8180,7.30,1.4,ml,11.0,154.64,0.119,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.0,0.460,5.0,automatic,nn,nn
2,2022-01-06 01:24:20.083000+00:00,61.3407,-149.8852,30.00,1.5,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.2,NaN,NaN,automatic,ak,ak
3,2022-01-06 01:15:15.851000+00:00,62.8703,-150.7625,111.30,2.1,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.8,NaN,NaN,automatic,ak,ak
4,2022-01-06 01:10:44.629000+00:00,64.0145,-148.7657,1.90,1.5,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.6,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17812,2021-12-23 03:08:37.279000+00:00,63.5339,-148.0043,1.60,0.7,ml,NaN,NaN,NaN,0.7700,...,2022-01-21T07:37:45.139Z,"49 km ENE of Cantwell, Alaska",earthquake,NaN,0.4,NaN,NaN,reviewed,ak,ak
17814,2021-12-23 03:02:42.557000+00:00,-57.3553,-25.8404,35.00,4.6,mb,NaN,74.00,6.742,0.7600,...,2022-01-19T03:08:29.040Z,South Sandwich Islands region,earthquake,13.8,2.0,0.125,19.0,reviewed,us,us
17816,2021-12-23 03:01:30.630000+00:00,5.8044,124.0274,517.23,4.3,mb,NaN,86.00,1.989,0.5000,...,2022-01-19T03:01:30.040Z,"Mindanao, Philippines",earthquake,8.9,7.3,0.142,14.0,reviewed,us,us
17817,2021-12-23 02:55:51.833000+00:00,-10.3928,166.1567,131.16,4.5,mb,NaN,136.00,5.126,0.7600,...,2022-01-19T02:52:12.040Z,"53 km NE of Lata, Solomon Islands",earthquake,13.1,10.4,0.192,8.0,reviewed,us,us


In [13]:
model_df['Unix Time'] = pd.to_datetime(model_df['time']).astype(int)/10**9
model_df.head(5)

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,Unix Time
0,2022-01-06 01:42:39.855000+00:00,37.3840,-117.1075,1.1,1.3,ml,8.0,149.46,0.875,0.2175,...,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.1,0.29,4.0,automatic,nn,nn,1.641433e+09
1,2022-01-06 01:31:33.698000+00:00,38.1810,-117.8180,7.3,1.4,ml,11.0,154.64,0.119,0.1017,...,"34 km SE of Mina, Nevada",earthquake,NaN,2.0,0.46,5.0,automatic,nn,nn,1.641433e+09
2,2022-01-06 01:24:20.083000+00:00,61.3407,-149.8852,30.0,1.5,ml,NaN,NaN,NaN,0.7500,...,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.2,NaN,NaN,automatic,ak,ak,1.641432e+09
3,2022-01-06 01:15:15.851000+00:00,62.8703,-150.7625,111.3,2.1,ml,NaN,NaN,NaN,0.3200,...,"41 km N of Petersville, Alaska",earthquake,NaN,0.8,NaN,NaN,automatic,ak,ak,1.641432e+09
4,2022-01-06 01:10:44.629000+00:00,64.0145,-148.7657,1.9,1.5,ml,NaN,NaN,NaN,0.7200,...,"17 km E of Ferry, Alaska",earthquake,NaN,0.6,NaN,NaN,automatic,ak,ak,1.641431e+09


In [14]:
for index, row in model_df.iterrows():
    if model_df.loc[index, 'type'] == 'earthquake':
        model_df.loc[index,'grouped_type'] = 1
    else:
        model_df.loc[index,'grouped_type'] = 0
model_df.head(5)

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,Unix Time,grouped_type
0,2022-01-06 01:42:39.855000+00:00,37.3840,-117.1075,1.1,1.3,ml,8.0,149.46,0.875,0.2175,...,earthquake,NaN,39.1,0.29,4.0,automatic,nn,nn,1.641433e+09,1.0
1,2022-01-06 01:31:33.698000+00:00,38.1810,-117.8180,7.3,1.4,ml,11.0,154.64,0.119,0.1017,...,earthquake,NaN,2.0,0.46,5.0,automatic,nn,nn,1.641433e+09,1.0
2,2022-01-06 01:24:20.083000+00:00,61.3407,-149.8852,30.0,1.5,ml,NaN,NaN,NaN,0.7500,...,earthquake,NaN,0.2,NaN,NaN,automatic,ak,ak,1.641432e+09,1.0
3,2022-01-06 01:15:15.851000+00:00,62.8703,-150.7625,111.3,2.1,ml,NaN,NaN,NaN,0.3200,...,earthquake,NaN,0.8,NaN,NaN,automatic,ak,ak,1.641432e+09,1.0
4,2022-01-06 01:10:44.629000+00:00,64.0145,-148.7657,1.9,1.5,ml,NaN,NaN,NaN,0.7200,...,earthquake,NaN,0.6,NaN,NaN,automatic,ak,ak,1.641431e+09,1.0


In [15]:
model_df.to_sql('earthquake_unix_grouptype', engine, if_exists='replace', index = False)

In [16]:
model_df.to_csv("csv/earthquake_unix_grouptype.csv",index=False)